In [1]:
import tkinter as tk
from tkinter import ttk
import mysql.connector
import re

def show_statistics():
    selected_region = combobox_region.get()
    
    if selected_region == "India":
        popup_title = "Indian Statistics"
        table_name = "india"
    elif selected_region == "Canada":
        popup_title = "Canadian Statistics"
        table_name = "canada"
    elif selected_region == "USA":
        popup_title = "USA Statistics"
        table_name = "united_states"
    elif selected_region == "Great Britain":
        popup_title = "Great Britain Statistics"
        table_name = "great_britain"
    else:
        popup_title = "Invalid Selection"
        popup_message = "Please select a valid region."

    popup_window = tk.Toplevel(root)
    popup_window.title(popup_title)
    
    # Labels
    filter_label = ttk.Label(popup_window, text="Filter:")
    filter_label.grid(row=0, column=0, padx=5, pady=5)
    
    # Combo Boxes
    filter_combobox_1 = ttk.Combobox(popup_window, values=[
        "trending_date", "category_id", "views", "likes", "dislikes", "comment_count"
    ])
    filter_combobox_1.grid(row=1, column=0, padx=5, pady=5)

    # Explore Button
    explore_button_popup = ttk.Button(popup_window, text="Explore", command=lambda: explore_selected_filter(filter_combobox_1.get(), table_name))
    explore_button_popup.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

def explore_selected_filter(selected_filter, table_name):
    if selected_filter == "trending_date":
        show_trending_date_popup(table_name)
    elif selected_filter == "category_id":
        show_category_popup(table_name)
    elif selected_filter in ['views', 'likes', 'dislikes', 'comment_count']:
        show_numeric_filter_popup(selected_filter, table_name)
    else:
        popup_window = tk.Toplevel(root)
        popup_window.title("Selected Filter")
        popup_message = f"The filter you chose is '{selected_filter}'"
        popup_label = ttk.Label(popup_window, text=popup_message)
        popup_label.pack(padx=10, pady=10)

def show_trending_date_popup(table_name):
    popup = tk.Toplevel(root)
    popup.title("Enter Trending Date")
    popup.geometry("300x150")
    
    # Labels and Entry for entering date and comparison operator
    comparison_label = ttk.Label(popup, text="Comparison Operator:")
    comparison_label.grid(row=0, column=0, padx=5, pady=5)
    comparison_entry = ttk.Entry(popup)
    comparison_entry.grid(row=0, column=1, padx=5, pady=5)
    
    date_label = ttk.Label(popup, text="Enter date:")
    date_label.grid(row=1, column=0, padx=5, pady=5)
    date_entry = ttk.Entry(popup)
    date_entry.grid(row=1, column=1, padx=5, pady=5)
    
    # Button to save the entered data
    save_button = ttk.Button(popup, text="Save", command=lambda: save_date(popup, comparison_entry.get(), date_entry.get(), table_name))
    save_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

def save_date(popup, comparison, date, table_name):
    global filter_var
    if not re.match(r"([><]=?)?\d{4}-\d{2}-\d{2}", date):
        show_error_popup(popup, "Please enter the date in the correct format (yyyy-mm-dd).")
        return
    
    # Connect to MySQL database
    connection = mysql.connector.connect(host="localhost",
                                         user="root",
                                         password="1408",
                                         database="youtube")
    cursor = connection.cursor()
    
    # Execute SQL query
    cursor.execute(f"SELECT {table_name}.title, {table_name}.channel, category.category_name, {table_name}.publish_time, {table_name}.trending_date, {table_name}.views, {table_name}.likes, {table_name}.dislikes, {table_name}.comment_count FROM category,{table_name} WHERE category.category_id = {table_name}.category AND trending_date {comparison} '{date}'")
    result = cursor.fetchall()
    
    # Display the result
    show_treeview(table_name, result)
    
    cursor.close()
    connection.close()

    popup.destroy()

# Similar modifications for other save functions

def show_category_popup(table_name):
    popup = tk.Toplevel(root)
    popup.title("Choose Category")
    
    # Radio buttons for choosing category
    categories = {
        1: "Film and Animation",
        2: "Autos and Vehicles",
        10: "Music",
        15: "Pets & Animals",
        17: "Sports",
        18: "Short Movies",
        19: "Travel & Events",
        20: "Gaming",
        21: "Videoblogging",
        22: "People & Blogs",
        23: "Comedy",
        24: "Entertainment",
        25: "News & Politics",
        26: "Howto & Style",
        27: "Education",
        28: "Science & Technology",
        30: "Movies",
        31: "Anime/Animation",
        32: "Action/Adventure",
        33: "Classics",
        35: "Documentary",
        36: "Drama",
        37: "Family",
        38: "Foreign",
        39: "Horror",
        40: "Sci-Fi/Fantasy",
        41: "Thriller",
        42: "Shorts",
        43: "Shows",
        44: "Trailers"
    }
    
    # Determine number of rows for radio buttons
    num_rows = (len(categories) + 1) // 2
    
    # Initialize row and column counters
    row = 0
    col = 0
    
    for cat_id, cat_name in categories.items():
        radio = ttk.Radiobutton(popup, text=cat_name, variable=category_var, value=cat_id)
        radio.grid(row=row, column=col, sticky=tk.W, padx=5, pady=5)
        col += 1
        if col % 2 == 0:
            row += 1
            col = 0
    
    # OK Button
    ok_button = ttk.Button(popup, text="OK", command=lambda: save_category(popup, table_name))
    ok_button.grid(row=row, column=0, columnspan=2, padx=5, pady=5)

def save_category(popup, table_name):
    category_id = category_var.get()
    
    # Connect to MySQL database
    connection = mysql.connector.connect(host="localhost",
                                         user="root",
                                         password="1408",
                                         database="youtube")
    cursor = connection.cursor()
    
    # Execute SQL query
    cursor.execute(f"SELECT * FROM {table_name} WHERE category = {category_id}")
    result = cursor.fetchall()
    
    # Display the result
    show_treeview(table_name, result)
    
    cursor.close()
    connection.close()

    popup.destroy()

def show_numeric_filter_popup(filter_type, table_name):
    popup = tk.Toplevel(root)
    popup.title(f"Enter {filter_type.capitalize()}")
    popup.geometry("300x150")
    
    # Labels and Entry for entering numeric value and comparison operator
    comparison_label = ttk.Label(popup, text="Comparison Operator:")
    comparison_label.grid(row=0, column=0, padx=5, pady=5)
    comparison_entry = ttk.Entry(popup)
    comparison_entry.grid(row=0, column=1, padx=5, pady=5)
    
    value_label = ttk.Label(popup, text=f"Enter {filter_type.capitalize()}:")
    value_label.grid(row=1, column=0, padx=5, pady=5)
    value_entry = ttk.Entry(popup)
    value_entry.grid(row=1, column=1, padx=5, pady=5)
    
    # Button to save the entered data
    save_button = ttk.Button(popup, text="Save", command=lambda: save_numeric_filter(popup, comparison_entry.get(), value_entry.get(), filter_type, table_name))
    save_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

def save_numeric_filter(popup, comparison, value, filter_type, table_name):
    if not value.isdigit():
        show_error_popup(popup, "Please enter a valid numeric value.")
        return
    
    # Connect to MySQL database
    connection = mysql.connector.connect(host="localhost",
                                         user="root",
                                         password="1408",
                                         database="youtube")
    cursor = connection.cursor()
    
    # Execute SQL query
    cursor.execute(f"SELECT * FROM {table_name} WHERE {filter_type} {comparison} {value}")
    result = cursor.fetchall()
    
    # Display the result
    show_treeview(table_name, result)
    
    cursor.close()
    connection.close()

    popup.destroy()

def show_treeview(table_name, data):
    treeview_popup = tk.Toplevel(root)
    treeview_popup.title(f"{table_name} Data")
    
    # Treeview columns
    columns = ["Title", "Channel", "Category", "Publish Time", "Trending Date", "Views", "Likes", "Dislikes", "Comment Count"]
    tree = ttk.Treeview(treeview_popup, columns=columns, show="headings")
    for col in columns:
        tree.heading(col, text=col)
    
    # Insert data into treeview
    for row in data:
        tree.insert("", tk.END, values=row)
    
    tree.pack(expand=True, fill="both")

def show_error_popup(popup, message):
    error_popup = tk.Toplevel(popup)
    error_popup.title("Error")
    error_label = ttk.Label(error_popup, text=message)
    error_label.pack(padx=10, pady=10)
    
    # OK Button
    ok_button = ttk.Button(error_popup, text="OK", command=error_popup.destroy)
    ok_button.pack(padx=10, pady=10)

root = tk.Tk()
root.title("YouTube Statistics")

# Main window widgets
label = ttk.Label(root, text="Select Region:")
label.pack(padx=10, pady=10)

combobox_region = ttk.Combobox(root, values=["India", "Canada", "USA", "Great Britain"])
combobox_region.pack(padx=10, pady=10)

explore_button = ttk.Button(root, text="Explore", command=show_statistics)
explore_button.pack(padx=10, pady=10)

category_var = tk.IntVar()

root.mainloop()